In [39]:
import numpy as np
import pandas as pd
import os
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import roc_auc_score

# Data

train_data.csv: базовый набор тренировочных признаков.

test_data.csv: базовый набор тестовых признаков.

train_targets.csv: результаты тренировочных игр (победа Radiat - 0/1).

train_raw_data.jsonl: сырые тренировочные данные.

test_raw_data.jsonl: сырые тестовые данные.

In [33]:
df_train_data = pd.read_csv('/kaggle/input/bi-ml-competition-2023/train_data.csv', 
                                    index_col='match_id_hash')
df_train_targets = pd.read_csv('/kaggle/input/bi-ml-competition-2023/train_targets.csv', 
                                   index_col='match_id_hash')
df_test_data = pd.read_csv('/kaggle/input/bi-ml-competition-2023/test_data.csv', 
                                   index_col='match_id_hash')

### EDA

let us look at the data and its statistics

In [9]:
df_train_data.info()

<class 'pandas.core.frame.DataFrame'>
Index: 31698 entries, b9c57c450ce74a2af79c9ce96fac144d to e2ca68ac1a6847f4a37f6c9c8ee8695b
Columns: 245 entries, game_time to d5_sen_placed
dtypes: float64(30), int64(215)
memory usage: 59.5+ MB


In [12]:
df_train_data.describe()

,game_time,game_mode,lobby_type,objectives_len,chat_len,r1_hero_id,r1_kills,r1_deaths,r1_assists,r1_denies,...,d5_stuns,d5_creeps_stacked,d5_camps_stacked,d5_rune_pickups,d5_firstblood_claimed,d5_teamfight_participation,d5_towers_killed,d5_roshans_killed,d5_obs_placed,d5_sen_placed
count,31698.000000,31698.000000,31698.000000,31698.000000,31698.000000,31698.000000,31698.000000,31698.000000,31698.000000,31698.000000,...,31698.000000,31698.000000,31698.000000,31698.000000,31698.000000,31698.000000,31698.000000,31698.000000,31698.000000,31698.000000
mean,1146.519496,19.583538,4.771563,6.535239,7.358982,51.081614,3.151713,3.273361,4.661998,6.255600,...,11.773022,1.021957,0.338381,4.697363,0.088965,0.416247,0.299640,0.024449,1.255568,0.780933
std,766.936655,6.306635,3.260899,6.486933,13.295763,34.573173,3.727116,3.281208,5.207887,8.144267,...,20.400131,3.427767,0.946140,4.662255,0.284697,0.267604,0.730835,0.172030,2.582681,2.436478
min,0.000000,2.000000,0.000000,0.000000,0.000000,1.000000,0.000000,0.000000,0.000000,0.000000,...,-3.294393,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
25%,521.000000,22.000000,0.000000,1.000000,0.000000,20.000000,0.000000,1.000000,1.000000,1.000000,...,0.000000,0.000000,0.000000,1.000000,0.000000,0.235294,0.000000,0.000000,0.000000,0.000000
50%,1044.000000,22.000000,7.000000,4.000000,3.000000,44.000000,2.000000,2.000000,3.000000,3.000000,...,1.532971,0.000000,0.000000,3.000000,0.000000,0.444444,0.000000,0.000000,0.000000,0.000000
75%,1659.000000,22.000000,7.000000,10.000000,9.000000,81.000000,5.000000,5.000000,7.000000,9.000000,...,16.129388,0.000000,0.000000,7.000000,0.000000,0.600000,0.000000,0.000000,1.000000,0.000000
max,4933.000000,23.000000,7.000000,43.000000,291.000000,120.000000,32.000000,27.000000,40.000000,83.000000,...,222.612630,107.000000,29.000000,57.000000,1.000000,2.000000,9.000000,5.000000,26.000000,47.000000


In [13]:
df_train_data.head()

,game_time,game_mode,lobby_type,objectives_len,chat_len,r1_hero_id,r1_kills,r1_deaths,r1_assists,r1_denies,...,d5_stuns,d5_creeps_stacked,d5_camps_stacked,d5_rune_pickups,d5_firstblood_claimed,d5_teamfight_participation,d5_towers_killed,d5_roshans_killed,d5_obs_placed,d5_sen_placed
match_id_hash,,,,,,,,,,,,,,,,,,,,,
b9c57c450ce74a2af79c9ce96fac144d,658,4,0,3,10,15,7,2,0,7,...,0.000000,0,0,0,0,0.00,0,0,0,0
6db558535151ea18ca70a6892197db41,21,23,0,0,0,101,0,0,0,0,...,0.000000,0,0,0,0,0.00,0,0,0,0
19c39fe2af2b547e48708ca005c6ae74,160,22,7,0,0,57,0,0,0,1,...,0.000000,0,0,0,0,0.00,0,0,0,0
c96d629dc0c39f0c616d1949938a6ba6,1016,22,0,1,0,119,0,3,3,5,...,8.264696,0,0,3,0,0.25,0,0,3,0
156c88bff4e9c4668b0f53df3d870f1b,582,22,7,2,2,12,3,1,2,9,...,15.762911,3,1,0,1,0.50,0,0,3,0


In [17]:
df_test_data.info()

<class 'pandas.core.frame.DataFrame'>
Index: 7977 entries, a400b8f29dece5f4d266f49f1ae2e98a to 9928dfde50efcbdb2055da23dcdbc101
Columns: 245 entries, game_time to d5_sen_placed
dtypes: float64(30), int64(215)
memory usage: 15.0+ MB


In [18]:
df_test_data.describe()

,game_time,game_mode,lobby_type,objectives_len,chat_len,r1_hero_id,r1_kills,r1_deaths,r1_assists,r1_denies,...,d5_stuns,d5_creeps_stacked,d5_camps_stacked,d5_rune_pickups,d5_firstblood_claimed,d5_teamfight_participation,d5_towers_killed,d5_roshans_killed,d5_obs_placed,d5_sen_placed
count,7977.000000,7977.000000,7977.000000,7977.000000,7977.000000,7977.000000,7977.000000,7977.000000,7977.000000,7977.000000,...,7977.000000,7977.000000,7977.000000,7977.000000,7977.000000,7977.000000,7977.000000,7977.00000,7977.000000,7977.000000
mean,1144.347499,19.589695,4.775480,6.483640,7.257114,51.188667,3.132631,3.250721,4.702645,6.424846,...,11.774526,1.106807,0.362041,4.630438,0.094772,0.414823,0.300489,0.02432,1.323806,0.792654
std,768.324114,6.298772,3.259522,6.512873,13.643925,34.723592,3.713196,3.291858,5.294377,8.436137,...,21.272956,3.928699,1.030529,4.566687,0.292919,0.267355,0.739074,0.16429,2.655274,2.443931
min,0.000000,2.000000,0.000000,0.000000,0.000000,1.000000,0.000000,0.000000,0.000000,0.000000,...,-6.191284,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.00000,0.000000,0.000000
25%,518.000000,22.000000,0.000000,1.000000,0.000000,20.000000,0.000000,1.000000,1.000000,1.000000,...,0.000000,0.000000,0.000000,1.000000,0.000000,0.250000,0.000000,0.00000,0.000000,0.000000
50%,1043.000000,22.000000,7.000000,4.000000,3.000000,44.000000,2.000000,2.000000,3.000000,3.000000,...,1.299609,0.000000,0.000000,3.000000,0.000000,0.440000,0.000000,0.00000,0.000000,0.000000
75%,1641.000000,22.000000,7.000000,10.000000,8.000000,81.000000,5.000000,5.000000,7.000000,9.000000,...,15.361314,0.000000,0.000000,7.000000,0.000000,0.588235,0.000000,0.00000,1.000000,0.000000
max,4574.000000,23.000000,7.000000,41.000000,207.000000,120.000000,31.000000,23.000000,37.000000,84.000000,...,277.618070,132.000000,26.000000,42.000000,1.000000,1.142857,8.000000,3.00000,25.000000,40.000000


In [21]:
df_test_data.head()

,game_time,game_mode,lobby_type,objectives_len,chat_len,r1_hero_id,r1_kills,r1_deaths,r1_assists,r1_denies,...,d5_stuns,d5_creeps_stacked,d5_camps_stacked,d5_rune_pickups,d5_firstblood_claimed,d5_teamfight_participation,d5_towers_killed,d5_roshans_killed,d5_obs_placed,d5_sen_placed
match_id_hash,,,,,,,,,,,,,,,,,,,,,
a400b8f29dece5f4d266f49f1ae2e98a,155,22,7,1,11,11,0,0,0,0,...,0.000000,0,0,0,0,0.000,0,0,0,0
46a0ddce8f7ed2a8d9bd5edcbb925682,576,22,7,1,4,14,1,0,3,1,...,8.664527,3,1,3,0,0.000,0,0,2,0
b1b35ff97723d9b7ade1c9c3cf48f770,453,22,7,1,3,42,0,1,1,0,...,0.000000,2,1,2,0,0.250,0,0,0,0
ab3cc6ccac661a1385e73a2e9f21313a,721,4,0,2,1,30,2,2,1,3,...,0.000000,0,0,1,0,0.125,0,0,0,0
54aaab1cb8cc5df3c253641618673266,752,22,7,1,0,8,2,0,2,8,...,0.766459,0,0,1,0,1.000,0,0,0,0


so, we have 31698 counts for train and 7977 for test data. Then let us look at the targets

In [14]:
df_train_targets.describe()

,game_time,duration,time_remaining
count,31698.000000,31698.000000,31698.000000
mean,1146.519496,2325.543410,1179.023913
std,766.936655,671.293146,769.720296
min,0.000000,901.000000,31.000000
25%,521.000000,1869.000000,551.000000
50%,1044.000000,2269.000000,1082.000000
75%,1659.000000,2746.000000,1692.000000
max,4933.000000,5638.000000,5108.000000


In [20]:
df_train_targets.head()

,game_time,radiant_win,duration,time_remaining,next_roshan_team
match_id_hash,,,,,
b9c57c450ce74a2af79c9ce96fac144d,658,True,1154,496,NaN
6db558535151ea18ca70a6892197db41,21,True,1503,1482,Radiant
19c39fe2af2b547e48708ca005c6ae74,160,False,2063,1903,NaN
c96d629dc0c39f0c616d1949938a6ba6,1016,True,2147,1131,Radiant
156c88bff4e9c4668b0f53df3d870f1b,582,False,1927,1345,Dire


In [15]:
df_train_targets.info()

<class 'pandas.core.frame.DataFrame'>
Index: 31698 entries, b9c57c450ce74a2af79c9ce96fac144d to e2ca68ac1a6847f4a37f6c9c8ee8695b
Data columns (total 5 columns):
 #   Column            Non-Null Count  Dtype 
---  ------            --------------  ----- 
 0   game_time         31698 non-null  int64 
 1   radiant_win       31698 non-null  bool  
 2   duration          31698 non-null  int64 
 3   time_remaining    31698 non-null  int64 
 4   next_roshan_team  20190 non-null  object
dtypes: bool(1), int64(3), object(1)
memory usage: 1.2+ MB


want to look at the statistics in victories and defeats

In [22]:
df_train_targets['radiant_win'].value_counts()

True     16670
False    15028
Name: radiant_win, dtype: int64

almost 50/50

as we can see from train/test data we have statistics from seperate players, I consider combine them, so we'll have teams' statistics bcs the victory can easier gained by whole team than separate players. Also I decided to merge two datasets just for this work

In [34]:
df_train = pd.merge(df_train_data, df_train_targets, left_index = True, right_index = True)

In [35]:
df_train['radiant_win'] = df_train['radiant_win'].astype('int')

In [26]:
df_train.head()

,game_time_x,game_mode,lobby_type,objectives_len,chat_len,r1_hero_id,r1_kills,r1_deaths,r1_assists,r1_denies,...,d5_teamfight_participation,d5_towers_killed,d5_roshans_killed,d5_obs_placed,d5_sen_placed,game_time_y,radiant_win,duration,time_remaining,next_roshan_team
match_id_hash,,,,,,,,,,,,,,,,,,,,,
b9c57c450ce74a2af79c9ce96fac144d,658,4,0,3,10,15,7,2,0,7,...,0.00,0,0,0,0,658,1,1154,496,NaN
6db558535151ea18ca70a6892197db41,21,23,0,0,0,101,0,0,0,0,...,0.00,0,0,0,0,21,1,1503,1482,Radiant
19c39fe2af2b547e48708ca005c6ae74,160,22,7,0,0,57,0,0,0,1,...,0.00,0,0,0,0,160,0,2063,1903,NaN
c96d629dc0c39f0c616d1949938a6ba6,1016,22,0,1,0,119,0,3,3,5,...,0.25,0,0,3,0,1016,1,2147,1131,Radiant
156c88bff4e9c4668b0f53df3d870f1b,582,22,7,2,2,12,3,1,2,9,...,0.50,0,0,3,0,582,0,1927,1345,Dire


In [29]:
def summa(df, column, array): 
    s = 0
    for i in array:
        s += df[i]
    df[column] = s
    for i in array:
        df.drop(i, inplace = True, axis = 1)

In [37]:
#radiant
summa(df_train, 'radiant_gold', ['r1_gold', 'r2_gold', 'r3_gold', 'r4_gold', 'r5_gold'])
summa(df_train, 'radiant_xp', ['r1_xp', 'r2_xp', 'r3_xp', 'r4_xp', 'r5_xp'])
summa(df_train, 'radiant_obs', ['r1_obs_placed', 'r2_obs_placed', 'r3_obs_placed', 'r4_obs_placed', 'r5_obs_placed'])
summa(df_train, 'radiant_sen', ['r1_sen_placed', 'r2_sen_placed', 'r3_sen_placed', 'r4_sen_placed','r5_sen_placed'])
summa(df_train, 'radiant_towers_killed', ['r1_towers_killed', 'r2_towers_killed', 'r3_towers_killed', 'r4_towers_killed','r5_towers_killed'])
summa(df_train, 'radiant_stuns', ['r1_stuns', 'r2_stuns', 'r3_stuns', 'r4_stuns','r5_stuns'])

df_train['radiant_wards'] = df_train['radiant_obs'] + df_train['radiant_sen']
df_train['radiant_gold_xp_ratio'] = df_train['radiant_gold']/df_train['radiant_xp']

In [36]:
#dire
summa(df_train, 'dire_gold', ['d1_gold', 'd2_gold', 'd3_gold', 'd4_gold', 'd5_gold'])
summa(df_train, 'dire_xp', ['d1_xp', 'd2_xp', 'd3_xp', 'd4_xp', 'd5_xp'])
summa(df_train, 'dire_obs', ['d1_obs_placed', 'd2_obs_placed', 'd3_obs_placed', 'd4_obs_placed', 'd5_obs_placed'])
summa(df_train, 'dire_sen', ['d1_sen_placed', 'd2_sen_placed', 'd3_sen_placed', 'd4_sen_placed','d5_sen_placed'])
summa(df_train, 'dire_towers_killed', ['d1_towers_killed', 'd2_towers_killed', 'd3_towers_killed', 'd4_towers_killed','d5_towers_killed'])
summa(df_train, 'dire_stuns', ['d1_stuns', 'd2_stuns', 'd3_stuns', 'd4_stuns','d5_stuns'])

df_train['dire_wards'] = df_train['dire_obs'] + df_train['dire_sen']
df_train['dire_gold_xp_ratio'] = df_train['dire_gold']/df_train['dire_xp']

In [38]:
df_train.head()

,game_time_x,game_mode,lobby_type,objectives_len,chat_len,r1_hero_id,r1_kills,r1_deaths,r1_assists,r1_denies,...,dire_wards,dire_gold_xp_ratio,radiant_gold,radiant_xp,radiant_obs,radiant_sen,radiant_towers_killed,radiant_stuns,radiant_wards,radiant_gold_xp_ratio
match_id_hash,,,,,,,,,,,,,,,,,,,,,
b9c57c450ce74a2af79c9ce96fac144d,658,4,0,3,10,15,7,2,0,7,...,7,1.016944,18172,17566,4,3,1,42.060296,7,1.034498
6db558535151ea18ca70a6892197db41,21,23,0,0,0,101,0,0,0,0,...,0,inf,880,0,0,0,0,0.000000,0,inf
19c39fe2af2b547e48708ca005c6ae74,160,22,7,0,0,57,0,0,0,1,...,0,0.733386,2947,2942,1,0,0,7.231550,1,1.001700
c96d629dc0c39f0c616d1949938a6ba6,1016,22,0,1,0,119,0,3,3,5,...,8,0.761457,23650,26116,6,2,0,63.184530,8,0.905575
156c88bff4e9c4668b0f53df3d870f1b,582,22,7,2,2,12,3,1,2,9,...,5,0.986763,12628,13044,2,0,0,5.332095,2,0.968108


In [41]:
#test data
summa(df_test_data, 'radiant_gold', ['r1_gold', 'r2_gold', 'r3_gold', 'r4_gold', 'r5_gold'])
summa(df_test_data, 'radiant_xp', ['r1_xp', 'r2_xp', 'r3_xp', 'r4_xp', 'r5_xp'])
summa(df_test_data, 'radiant_obs', ['r1_obs_placed', 'r2_obs_placed', 'r3_obs_placed', 'r4_obs_placed', 'r5_obs_placed'])
summa(df_test_data, 'radiant_sen', ['r1_sen_placed', 'r2_sen_placed', 'r3_sen_placed', 'r4_sen_placed','r5_sen_placed'])
summa(df_test_data, 'radiant_towers_killed', ['r1_towers_killed', 'r2_towers_killed', 'r3_towers_killed', 'r4_towers_killed','r5_towers_killed'])
summa(df_test_data, 'radiant_stuns', ['r1_stuns', 'r2_stuns', 'r3_stuns', 'r4_stuns','r5_stuns'])

df_test_data['radiant_wards'] = df_test_data['radiant_obs'] + df_test_data['radiant_sen']
df_test_data['radiant_gold_xp_ratio'] = df_test_data['radiant_gold']/df_test_data['radiant_xp']

In [43]:
summa(df_test_data, 'dire_gold', ['d1_gold', 'd2_gold', 'd3_gold', 'd4_gold', 'd5_gold'])
summa(df_test_data, 'dire_xp', ['d1_xp', 'd2_xp', 'd3_xp', 'd4_xp', 'd5_xp'])
summa(df_test_data, 'dire_obs', ['d1_obs_placed', 'd2_obs_placed', 'd3_obs_placed', 'd4_obs_placed', 'd5_obs_placed'])
summa(df_test_data, 'dire_sen', ['d1_sen_placed', 'd2_sen_placed', 'd3_sen_placed', 'd4_sen_placed','d5_sen_placed'])
summa(df_test_data, 'dire_towers_killed', ['d1_towers_killed', 'd2_towers_killed', 'd3_towers_killed', 'd4_towers_killed','d5_towers_killed'])
summa(df_test_data, 'dire_stuns', ['d1_stuns', 'd2_stuns', 'd3_stuns', 'd4_stuns','d5_stuns'])

df_test_data['dire_wards'] = df_test_data['dire_obs'] + df_test_data['dire_sen']
df_test_data['dire_gold_xp_ratio'] = df_test_data['dire_gold']/df_test_data['dire_xp']

# Model

I decided to build LogReg model and look at the results

In [51]:
columns = ['radiant_gold', 'dire_gold',
           'radiant_xp', 'dire_xp',
          'radiant_gold_xp_ratio', 'dire_gold_xp_ratio',
          'radiant_wards', 'dire_wards',
          'radiant_towers_killed', 'dire_towers_killed',
          'radiant_stuns', 'dire_stuns']

In [66]:
df_train = df_train.replace([np.inf, -np.inf], np.nan).fillna(0)
df_test_data = df_test_data.replace([np.inf, -np.inf], np.nan).fillna(0).filter(columns, axis=1)

In [58]:
X_train = df_train.filter(columns, axis=1).dropna()
y_train = df_train['radiant_win'].dropna()

In [59]:
model =  LogisticRegression()
model.fit(X_train, y_train)

LogisticRegression()

In [68]:
submission = pd.DataFrame()
submission['match_id_hash'] = df_test_data.index
submission['radiant_win_prob']= model.predict_proba(df_test_data)[:, 1]
submission.to_csv('submission.csv', index=False)